<a href="https://colab.research.google.com/github/kunal266/rl_gym/blob/main/custom_env_rl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Install Dependencies

In [1]:
!pip install tensorflow==2.3.0
!pip install gym
!pip install keras
!pip install keras-rl2

# 1. Test Random Environment with OpenAI Gym

In [2]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random

In [11]:
class ShowerEnv(Env):
    def __init__(self):
        #actions we can take up,down,stay
        self.action_space = Discrete(3)
        # temperature array
        self.observation_space = Box(low=np.array([0]),high=np.array([100]))
        #set start temp
        self.state = 38 + random.randint(-3,3)
        # set shower length
        self.shower_length=60
    def step(self,action):
        #apply action
        self.state +=action -1
        #reduce shower length by 1
        self.shower_length -=1

        if self.state>=37 and self.state <=39:
            reward=1
        else:
            reward = -1
        #check if shower is running
        if self.shower_length <=0:
            done=True
        else:
            done=False   
        self.state +=random.randint(-1,1)
        info = {}
        return self.state,reward,done,info
    def render(self):
        pass
    def reset(self):
        self.state = 38+random.randint(-3,3)
        self.shower_length = 60
        return self.state


In [12]:
env = ShowerEnv()

/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [19]:
# env = gym.make('CartPole-v1')
states = env.observation_space.shape
actions = env.action_space.n

In [14]:
actions

3

In [15]:
env.action_space.sample()

2

In [17]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        # env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print(f'Episode:{episode} Score:{score}')

Episode:1 Score:-34
Episode:2 Score:6
Episode:3 Score:-30
Episode:4 Score:-40
Episode:5 Score:-36
Episode:6 Score:-30
Episode:7 Score:-60
Episode:8 Score:4
Episode:9 Score:-46
Episode:10 Score:-6


# 2. Create a Deep Learning Model with Keras

In [20]:
states

(1,)

In [18]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [21]:
def build_model(states, actions):
    model = Sequential()
    # model.add(Flatten(input_shape=(1,states)))

    model.add(Dense(24, activation='relu',input_shape = states))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [27]:
del model

In [28]:
model = build_model(states, actions)

In [29]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 24)                48        
_________________________________________________________________
dense_4 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 75        
Total params: 723
Trainable params: 723
Non-trainable params: 0
_________________________________________________________________


# 3. Build Agent with Keras-RL

In [24]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [30]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [31]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
    1/10000 [..............................] - ETA: 9:06 - reward: -1.0000

/usr/local/lib/python3.7/dist-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 82s 8ms/step - reward: -0.6910
166 episodes - episode_reward: -41.542 [-60.000, 20.000] - loss: 2.073 - mae: 9.754 - mean_q: -10.250

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 81s 8ms/step - reward: -0.6214
167 episodes - episode_reward: -37.246 [-60.000, 26.000] - loss: 3.201 - mae: 13.163 - mean_q: -19.017

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 81s 8ms/step - reward: -0.6532
167 episodes - episode_reward: -39.162 [-60.000, 36.000] - loss: 3.564 - mae: 14.331 - mean_q: -20.730

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 85s 8ms/step - reward: -0.6784
166 episodes - episode_reward: -40.627 [-60.000, 28.000] - loss: 3.264 - mae: 13.618 - mean_q: -19.624

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 87s 9ms/step - reward: -0.6566
done, took 415.318 seconds


In [32]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: -60.000, steps: 60
Episode 2: reward: -60.000, steps: 60
Episode 3: reward: -46.000, steps: 60
Episode 4: reward: -50.000, steps: 60
Episode 5: reward: -54.000, steps: 60
Episode 6: reward: -50.000, steps: 60
Episode 7: reward: -60.000, steps: 60
Episode 8: reward: -56.000, steps: 60
Episode 9: reward: -60.000, steps: 60
Episode 10: reward: -60.000, steps: 60
Episode 11: reward: -60.000, steps: 60
Episode 12: reward: -58.000, steps: 60
Episode 13: reward: -60.000, steps: 60
Episode 14: reward: -60.000, steps: 60
Episode 15: reward: -60.000, steps: 60
Episode 16: reward: -54.000, steps: 60
Episode 17: reward: -58.000, steps: 60
Episode 18: reward: -60.000, steps: 60
Episode 19: reward: -56.000, steps: 60
Episode 20: reward: -60.000, steps: 60
Episode 21: reward: -60.000, steps: 60
Episode 22: reward: -56.000, steps: 60
Episode 23: reward: -52.000, steps: 60
Episode 24: reward: -60.000, steps: 60
Episode 25: reward: -54.000, steps: 60
Episo

In [34]:
_ = dqn.test(env, nb_episodes=15, visualize=False)

Testing for 15 episodes ...
Episode 1: reward: -60.000, steps: 60
Episode 2: reward: -60.000, steps: 60
Episode 3: reward: -58.000, steps: 60
Episode 4: reward: -60.000, steps: 60
Episode 5: reward: -52.000, steps: 60
Episode 6: reward: -60.000, steps: 60
Episode 7: reward: -58.000, steps: 60
Episode 8: reward: -60.000, steps: 60
Episode 9: reward: -60.000, steps: 60
Episode 10: reward: -60.000, steps: 60
Episode 11: reward: -56.000, steps: 60
Episode 12: reward: -60.000, steps: 60
Episode 13: reward: -60.000, steps: 60
Episode 14: reward: -60.000, steps: 60
Episode 15: reward: -56.000, steps: 60


# 4. Reloading Agent from Memory

In [ ]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)

In [ ]:
del model
del dqn
del env

In [ ]:
env = gym.make('CartPole-v0')
actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [ ]:
dqn.load_weights('dqn_weights.h5f')

In [ ]:
_ = dqn.test(env, nb_episodes=5, visualize=True)